In [1]:
using Distributions
using Interpolations
using Distributed
using DataFrames
using Printf
using JLD
using CSV
using Dierckx
using Dates
# using PyPlot
# using Seaborn

main_path = "/home/artur/BondPricing"
module_path = string(main_path, "/", "Julia/modules/")
modls = ["ModelObj", "AnalyticFunctions", 
         "BondPrInterp", "EqFinDiff", 
         "Batch", "ModelPlots","FullInfoEq", "JointEq"]
for modl in modls
    include(string(joinpath(module_path, modl), ".jl"))
end

ENV["LINES"] = 750
ENV["COLUMNS"] =999

┌ Info: Recompiling stale cache file /home/artur/.julia/compiled/v1.2/EqFinDiff.ji for EqFinDiff [top-level]
└ @ Base loading.jl:1240
┌ Info: Recompiling stale cache file /home/artur/.julia/compiled/v1.2/Batch.ji for Batch [top-level]
└ @ Base loading.jl:1240
┌ Info: Recompiling stale cache file /home/artur/.julia/compiled/v1.2/FullInfoEq.ji for FullInfoEq [top-level]
└ @ Base loading.jl:1240


PyPlot.Figure(PyObject <Figure size 400x100 with 1 Axes>)

999

In [10]:
for modl in modls
    include(string(joinpath(module_path, modl), ".jl"))
end

PyPlot.Figure(PyObject <Figure size 400x100 with 1 Axes>)

In [2]:
mu_s = .5
kep = 25 * 10^-4
kotc = 50 * 10^-4
s_iota = 1e-4
r_iota = 0.0
r_lambda = .2
r_sigmah = .25
ep_m = 1.
ep_c = 3.
ep_p = 35.
otc_m = 1.

jep = JointEq.store_joint_eq_parameters(mu_s, kep, kotc;
                                        s_iota=s_iota,
                                        r_iota=r_iota,
                                        r_lambda=r_lambda,
                                        r_sigmah=r_sigmah)

# Check for missing parameters
# Lambda and sigmah can be NaN => CVM model
missing = false
if any([isnan(getfield(jep, x)) for x in [:mu_s, :kep, :kotc]])
    println("Missing Market Parameters!")
    missing = true
end
if any([isnan(getfield(jep.sfp, :iota)) for x in fieldnames(JointEq.FirmSpecificParams)])
    println("Missing Safe Firm's specific parameters!")
    missing = true
end
if any([isnan(getfield(jep.rfp, :iota)) for x in fieldnames(JointEq.FirmSpecificParams)])
    println("Missing Risky Firm's specific parameters!")
    missing = true
end

# #################################################################
# Safe and Risky Firms' Models ####################################
# #################################################################
cbt = Batch.get_bt(;comb_num=1, model="cvm")
sbt = Batch.get_bt(;comb_num=1, model="svm")

# Identify Firm Model
sf_model = .&(!isnan(jep.sfp.iota),!isnan(jep.sfp.lambda)) ? "svm" : "cvm"
rf_model = .&(!isnan(jep.rfp.iota),!isnan(jep.rfp.lambda)) ? "svm" : "cvm"

# Set Models
sf_bt = sf_model == "cvm" ? cbt : sbt
rf_bt = rf_model == "cvm" ? cbt : sbt
# #################################################################   

Setting parameter dictionary using unique combination ID...
Setting parameter dictionary using unique combination ID...


Main.Batch.BatchStruct
  model: String "svm"
  dfn: Main.Batch.DirFileNames
  dfc: Main.Batch.BatchDFColumns
  mi: Main.Batch.ModelInputs
  bp: Main.Batch.BatchParams
  coupon_grid: Array{Float64}((22,)) [0.5, 1.0, 1.5, 2.0, 2.5, 3.0, 3.5, 4.0, 4.5, 5.0  …  6.5, 7.0, 7.5, 8.0, 8.5, 9.0, 9.5, 10.0, 10.5, 11.0]
  pvb_tol_vec: Array{Float64}((4,)) [1.0000000000000004e-6, 1.0000000000000004e-5, 0.00010000000000000003, 0.0010000000000000005]


In [3]:
jeq= JointEq.ep_constructor(jep, sf_bt, rf_bt;
                        ep_m=ep_m,
                        ep_c=ep_c,
                        ep_p=ep_p,
                        run_pool_eq=false,
                        run_sep_eq=false,                       
                        sf_obj_fun=:firm_value,
                        rf_obj_fun=:MBR,
                        rerun_full_info=true,
                        rerun_pool=false,
                        rerun_sep=false)

Setting parameter dictionary to batch object's parameter dictionary
Setting parameter values... 
Setting parameter dictionary using unique combination ID...
Setting initial vbl value to NaN
Setting initial vbh value to NaN
Setting initial c value to NaN
Setting initial p value to NaN
Constant Volatility Model: setting vbh to vbl, lambda to NaN
Setting parameter dictionary to batch object's parameter dictionary
Setting parameter values... 
Setting parameter dictionary using unique combination ID...
Loading SVM object...
Batch object file not found.
Batch Object is incompatible! Loading surfaces only instead.
Setting initial vbl value to NaN
Setting initial vbh value to NaN
Setting initial c value to NaN
Setting initial p value to NaN
Interpolating Bond Pricing Surfaces...
  4.636821 seconds (10.51 M allocations: 624.892 MiB, 8.01% gc time)
Batch object exists: true
Interpolating bond pricing surfaces...
  0.231902 seconds (811.45 k allocations: 137.112 MiB, 24.40% gc time)
COUNT: 0
COUN

debt_abs_per_diff: NaN
eq_abs_per_diff: NaN
Equity Vmax: 131.25
 
eq_max: 58.94943935035103
eq_vbl: 0.0
Shape of Gamma matrix: (1498,)
Computing equity values... 
V0: 100.0; equity: 8.923001255309016
Equity Core Function Computation Time: 0.04413890838623047
Total Equity FD Core Function Computation Time: 0.10550999641418457
Total computation time: 0.11140584945678711
Computing Equity Vmax
Vmax: 125.0
phi0: -67.2250452521036
phi1: 0.9949999999999999
pv_rfdebt: 90.54902824148483
debt_abs_per_diff: NaN
eq_abs_per_diff: NaN
Equity Vmax: 131.25
 
eq_max: 52.95920800107071
eq_vbl: 0.0
Shape of Gamma matrix: (1498,)
Computing equity values... 
V0: 100.0; equity: 2.419508354041986
Equity Core Function Computation Time: 0.03882789611816406
Total Equity FD Core Function Computation Time: 0.22718501091003418
Total computation time: 0.23297905921936035
Computing Equity Vmax
Vmax: 125.0
phi0: -70.4140127680077
phi1: 0.9949999999999999
pv_rfdebt: 94.84442006421837
debt_abs_per_diff: NaN
eq_abs_per_

Computing equity values... 
V0: 100.0; equity: 86.78567401264381
Equity Core Function Computation Time: 0.04061388969421387
Total Equity FD Core Function Computation Time: 0.0865318775177002
Total computation time: 0.10800814628601074
Computing Equity Vmax
Vmax: 125.0
phi0: -13.012597481734112
phi1: 0.9999999999999998
pv_rfdebt: 17.527367255014916
debt_abs_per_diff: 0.0005605785217429761
eq_abs_per_diff: 0.000338309868025438
Equity Vmax: 213.7924197645393
 
eq_max: 200.77980902412108
eq_vbl: 0.0
Computing Constant Volatility Equity Values
Finished computing Constant Volatility Equity Values
Time to compute Constant Volatility Equity Values: 0.008093118667602539
 
Shape of Gamma matrix: (1498,)
Computing equity values... 
V0: 100.0; equity: 86.78566835476465
Equity Core Function Computation Time: 0.04278302192687988
Total Equity FD Core Function Computation Time: 0.09131598472595215
Total computation time: 0.11252403259277344
Computing Equity Vmax
Vmax: 125.0
phi0: -13.012597481734112
p

V0: 100.0; equity: 80.12978233281548
Equity Core Function Computation Time: 0.09125089645385742
Total Equity FD Core Function Computation Time: 0.14810490608215332
Total computation time: 0.17218685150146484
Computing Equity Vmax
Vmax: 125.0
phi0: -19.17645944676606
phi1: 0.9999999999999998
pv_rfdebt: 25.829804375811452
debt_abs_per_diff: 6.243631213931801e-5
eq_abs_per_diff: 0.000856451865232897
Equity Vmax: 235.70614279040458
 
eq_max: 216.5295840375969
eq_vbl: 0.0
Computing Constant Volatility Equity Values
Finished computing Constant Volatility Equity Values
Time to compute Constant Volatility Equity Values: 0.008053064346313477
 
Shape of Gamma matrix: (1498,)
Computing equity values... 
V0: 100.0; equity: 80.12585539791263
Equity Core Function Computation Time: 0.043643951416015625
Total Equity FD Core Function Computation Time: 0.20569300651550293
Total computation time: 0.22980284690856934
Computing Equity Vmax
Vmax: 125.0
phi0: -19.17645944676606
phi1: 0.9999999999999998
pv_rf

Computing equity values... 
V0: 100.0; equity: 72.97125671741043
Equity Core Function Computation Time: 0.039595842361450195
Total Equity FD Core Function Computation Time: 0.22924304008483887
Total computation time: 0.25959110260009766
Computing Equity Vmax
Vmax: 125.0
phi0: -25.340321411798005
phi1: 0.9999999999999998
pv_rfdebt: 34.132241496607996
debt_abs_per_diff: 5.27596080502767e-7
eq_abs_per_diff: 0.0009577266304370098
Equity Vmax: 300.8274042113857
 
eq_max: 275.486923357585
eq_vbl: 0.0
Computing Constant Volatility Equity Values
Finished computing Constant Volatility Equity Values
Time to compute Constant Volatility Equity Values: 0.00801706314086914
 
Shape of Gamma matrix: (1498,)
Computing equity values... 
V0: 100.0; equity: 72.95167550641028
Equity Core Function Computation Time: 0.038146018981933594
Total Equity FD Core Function Computation Time: 0.09167313575744629
Total computation time: 0.12398195266723633
Computing Equity Vmax
Vmax: 125.0
phi0: -25.340321411798005
ph

...

Computing equity values... 
V0: 100.0; equity: 0.0
Equity Core Function Computation Time: 0.03689098358154297
Total Equity FD Core Function Computation Time: 0.08094596862792969
Total computation time: 0.1385819911956787
Computing Equity Vmax
Vmax: 125.0
phi0: -70.25233305186774
phi1: 0.9999999999999998
pv_rfdebt: 94.62664496078884
debt_abs_per_diff: 1.540223536527622e-7
eq_abs_per_diff: 0.0009427598806476266
Equity Vmax: 838.0938943005542
 
eq_max: 767.8411311811699
eq_vbl: 0.0
Computing Constant Volatility Equity Values
Finished computing Constant Volatility Equity Values
Time to compute Constant Volatility Equity Values: 0.008260011672973633
 
Shape of Gamma matrix: (1498,)
Computing equity values... 
V0: 100.0; equity: 0.0
Equity Core Function Computation Time: 0.04226207733154297
Total Equity FD Core Function Computation Time: 0.08613014221191406
Total computation time: 0.143845796585083
Computing Equity Vmax
Vmax: 125.0
phi0: -70.25233305186774
phi1: 0.9999999999999998
pv_rfdebt:

Computing Constant Volatility Equity Values
Finished computing Constant Volatility Equity Values
Time to compute Constant Volatility Equity Values: 0.008267879486083984
 
Shape of Gamma matrix: (1498,)
Computing equity values... 
V0: 100.0; equity: 57.15709687265795
Equity Core Function Computation Time: 0.06526899337768555
Total Equity FD Core Function Computation Time: 0.10478806495666504
Total computation time: 0.2621190547943115
Computing Equity Vmax
Vmax: 125.0
phi0: -40.12615643600793
phi1: 0.9999999999999998
pv_rfdebt: 54.04807774722361
debt_abs_per_diff: 1.5402280486351664e-7
eq_abs_per_diff: 0.0008745093066596339
Equity Vmax: 490.01614230733185
 
eq_max: 449.8897703788332
eq_vbl: 0.0
Computing Constant Volatility Equity Values
Finished computing Constant Volatility Equity Values
Time to compute Constant Volatility Equity Values: 0.008289098739624023
 
Shape of Gamma matrix: (1498,)
Computing equity values... 
V0: 100.0; equity: 52.4825690779939
Equity Core Function Computation

debt_abs_per_diff: 1.548457398337978e-7
eq_abs_per_diff: 0.0009636396152546964
Equity Vmax: 383.9404694897265
 
eq_max: 351.53693919907107
eq_vbl: 0.0
Computing Constant Volatility Equity Values
Finished computing Constant Volatility Equity Values
Time to compute Constant Volatility Equity Values: 0.007986068725585938
 
Shape of Gamma matrix: (1498,)
Computing equity values... 
V0: 100.0; equity: 63.88989098885501
Equity Core Function Computation Time: 0.06505107879638672
Total Equity FD Core Function Computation Time: 0.10429096221923828
Total computation time: 0.2567250728607178
Computing Equity Vmax
Vmax: 125.0
phi0: -32.40332421073987
phi1: 0.9999999999999998
pv_rfdebt: 43.645779754747885
debt_abs_per_diff: 1.548457398337978e-7
eq_abs_per_diff: 0.0009636396152546964
Equity Vmax: 383.9404694897265
 
eq_max: 351.5369391990709
eq_vbl: 0.0
Computing Constant Volatility Equity Values
Finished computing Constant Volatility Equity Values
Time to compute Constant Volatility Equity Values: 

V0: 100.0; equity: 63.72577679139481
Equity Core Function Computation Time: 0.03693509101867676
Total Equity FD Core Function Computation Time: 0.08069992065429688
Total computation time: 0.11799383163452148
Computing Equity Vmax
Vmax: 125.0
phi0: -32.40332421073987
phi1: 0.9999999999999998
pv_rfdebt: 43.645779754747885
debt_abs_per_diff: 1.548457398337978e-7
eq_abs_per_diff: 0.0009636396152546964
Equity Vmax: 383.9404694897265
 
eq_max: 351.5369391990709
eq_vbl: 0.0
Computing Constant Volatility Equity Values
Finished computing Constant Volatility Equity Values
Time to compute Constant Volatility Equity Values: 0.008121013641357422
 
Shape of Gamma matrix: (1498,)
Computing equity values... 
V0: 100.0; equity: 63.66259496331999
Equity Core Function Computation Time: 0.04259800910949707
Total Equity FD Core Function Computation Time: 0.08589911460876465
Total computation time: 0.12320899963378906
Computing Equity Vmax
Vmax: 125.0
phi0: -32.40332421073987
phi1: 0.9999999999999998
pv_rfd

Main.JointEq.EPStruct(Main.JointEq.JointFirms(Main.JointEq.JointKStruct(0.5, 1.245075176429021, 1.0, 3.0, 35.0, 48.43065326479742, NaN, NaN, NaN, NaN), ModelObj.Firm
  mu_b: Float64 1.0
  m: Float64 1.0
  c: Float64 NaN
  p: Float64 NaN
  vbl: Float64 NaN
  vbh: Float64 NaN
  pm: ModelObj.FirmParams
  model: String "cvm"
  bi: ModelObj.BPrInputs
  bs: ModelObj.BPrSurfs
  bf: ModelObj.BPrInterpFuns
  bit: ModelObj.BPrFixedTTMInputs
  bft: ModelObj.BPrInterpFuns
  optKS: ModelObj.KStruct
, ModelObj.Firm
  mu_b: Float64 1.0
  m: Float64 1.0
  c: Float64 NaN
  p: Float64 NaN
  vbl: Float64 NaN
  vbh: Float64 NaN
  pm: ModelObj.FirmParams
  model: String "svm"
  bi: ModelObj.BPrInputs
  bs: ModelObj.BPrSurfs
  bf: ModelObj.BPrInterpFuns
  bit: ModelObj.BPrFixedTTMInputs
  bft: ModelObj.BPrInterpFuns
  optKS: ModelObj.KStruct
, Batch.BatchStruct
  model: String "cvm"
  dfn: Batch.DirFileNames
  dfc: Batch.BatchDFColumns
  mi: Batch.ModelInputs
  bp: Batch.BatchParams
  coupon_grid: Array{Flo

In [4]:
# INPUTS ###################################################
# Measure of Safe Firms
mu_s = .2

# Transaction Costs and Volatility Risk Parameters
cvmdict = Dict{Symbol,Array{Float64,1}}(:sigmal => [0.15],
                                        :m  => [1.],
                                        :gross_delta => [0.02],
                                        :kappa  => [25 * 1e-4],
                                        :mu_b => [1.0],
                                        :xi => [1.0],
                                        :iota => [x for x in Batch.cvm_param_values_dict[:iota] 
                                                  if .&(x >= 2.5 * 1e-4, x <= 20. * 1e-4)])
svmdict = deepcopy(cvmdict)
svmdict[:lambda] = [.2]
svmdict[:iota] = [.0]
svmdict[:sigmah] = Batch.svm_param_values_dict[:sigmah]
# #########################################################


# Get Safe and Risky Firms' Full Info Optimal Results #####
firm_obj_fun = :firm_value 
cvmdf, svmdf, _ = ModelPlots.get_cvm_svm_dfs(cvmdict, svmdict;
                                             firm_obj_fun=firm_obj_fun)
# #########################################################


# Form Safe Firm ##########################################
sf_model = "cvm"
sf_iota = 2.5 * 1e-4
sf_df = (sf_model == "cvm") ? cvmdf : svmdf
sf_comb_num = sf_df[abs.(sf_df[:, :iota] .- sf_iota) .< 1e-5, :comb_num][1]
sf_bt, sf = Batch.get_bt_mobj(; model=sf_model, comb_num=sf_comb_num)
sf = ModelObj.set_opt_k_struct(sf, sf_df)
# #########################################################


# Form Risky Firm #########################################
rf_model = "svm"
rf_comb_num = svmdf[1, :comb_num]
rf_bt, rf = Batch.get_bt_svm(; comb_num=rf_comb_num)
rf_df = (rf_model == "cvm") ? cvmdf : svmdf
rf = ModelObj.set_opt_k_struct(rf, rf_df)
# #########################################################


# #########################################################
# Optimal p/c Ratio: sf.optKS.p/sf.optKS.c ~~ 12
ep_c = JointEq.round_value(sf.optKS.c)
ep_ratio = JointEq.round_value(sf.optKS.p/sf.optKS.c)
ep_p = ep_ratio * ep_c

# Pick Capital Structure
jks = JointEq.JointKStruct(mu_s, 
                           sf.optKS.mu_b,
                           sf.optKS.m, ep_c, ep_p, 
                           NaN, NaN, NaN, NaN, NaN)

# Joint Firm Object
jf = JointEq.JointFirms(jks, sf, rf,
                        sf_bt,
                        rf_bt,  
                        cvmdf, svmdf)
# #########################################################


# Make Directories and File Names ##########################
# Make results directories
jks_fpath = JointEq.make_jeq_jks_fpath(jf)

# Full Information Equilibrium
fi_fpath_name = JointEq.get_jeq_mus_fname(jks_fpath; eq_type="full_info")

# Misrepresentation
misrep_fpath_name = JointEq.get_jeq_mus_fname(jks_fpath; eq_type="misrep")

# Pooling Equilibrium
pool_fpath_name = JointEq.get_jeq_mus_fname(jks_fpath; eq_type="pooling", mu_s=jf.jks.mu_s)

# Separating Equilibrium
sep_fpath_name = JointEq.get_jeq_mus_fname(jks_fpath; eq_type="separating")
# #########################################################


# Joint Equilibrium Parameters ############################
jep = JointEq.store_joint_eq_parameters(mu_s, sf.pm.kappa, sf.pm.kappa;
                                        s_iota=sf.pm.iota,
                                        s_lambda=sf.pm.lambda,
                                        s_sigmah=sf.pm.sigmah,
                                        r_iota=rf.pm.iota,
                                        r_lambda=rf.pm.lambda,
                                        r_sigmah=rf.pm.sigmah)
# #########################################################

Loading optimal results dataframe...
Setting parameter dictionary using (m, m_comb_num) ID pair...
Loading optimal results dataframe...
Setting parameter dictionary using (m, m_comb_num) ID pair...
Setting parameter dictionary using (m, m_comb_num) ID pair...
Setting parameter dictionary using (m, m_comb_num) ID pair...
Setting parameter dictionary using (m, m_comb_num) ID pair...
Setting parameter dictionary using unique combination ID...
Setting initial vbl value to NaN
Setting initial vbh value to NaN
Setting initial c value to NaN
Setting initial p value to NaN
Constant Volatility Model: setting vbh to vbl, lambda to NaN
Setting parameter dictionary using unique combination ID...
Loading SVM object...
Batch object file not found.
Batch Object is incompatible! Loading surfaces only instead.
Setting initial vbl value to NaN
Setting initial vbh value to NaN
Setting initial c value to NaN
Setting initial p value to NaN
Interpolating Bond Pricing Surfaces...
  0.212343 seconds (811.45 k

Main.JointEq.JointEqParams(0.2, 0.0025, 0.0025, Main.JointEq.FirmSpecificParams(0.00025, NaN, 0.15, NaN), Main.JointEq.FirmSpecificParams(0.0, 0.2, 0.15, 0.2), Main.JointEq.FirmCommonParams(100.0, 0.6, 0.27, 0.08, 0.02, 1.0))

In [55]:
cvmdf

,comb_num,m,m_comb_num,obj_fun,gross_delta,delta,iota,kappa,lambda,sigmah,sigmal,mu_b,c,p,vb,cvml_vb,cvmh_vb,debt_diff,eq_deriv,eq_min_val,sg_debt,debt,sg_equity,equity,firm_value,leverage,MBR,V0,r,alpha,pi,xi,abs_debt_diff,debt_per_diff,abs_debt_per_diff,eq_vb
,Int64,Float64,Int64,String,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64
1,21,1.0,21,firm_value,0.02,0.01975,0.00025,0.0025,NaN,NaN,0.15,1.0,4.63104,56.1283,59.9775,NaN,NaN,-0.000270284,NaN,NaN,56.1283,56.1281,54.3821,54.382,110.51,50.79,23.9563,100.0,0.08,0.6,0.27,1.0,0.000270284,-4.81546e-6,4.81546e-6,1.49214e-12
2,25,1.0,25,firm_value,0.02,0.0195,0.0005,0.0025,NaN,NaN,0.15,1.0,4.60654,55.8275,60.1019,NaN,NaN,-0.000138541,NaN,NaN,55.8276,55.8274,53.3787,53.3795,109.207,51.1208,20.8429,100.0,0.08,0.6,0.27,1.0,0.000138541,-2.48158e-6,2.48158e-6,4.47642e-13
3,29,1.0,29,firm_value,0.02,0.01925,0.00075,0.0025,NaN,NaN,0.15,1.0,4.58448,55.5602,60.2648,NaN,NaN,-0.000168214,NaN,NaN,55.5602,55.56,52.3421,52.3432,107.903,51.4906,17.7839,100.0,0.08,0.6,0.27,1.0,0.000168214,-3.02759e-6,3.02759e-6,3.90799e-14
4,33,1.0,33,firm_value,0.02,0.019,0.001,0.0025,NaN,NaN,0.15,1.0,4.56121,55.2789,60.415,NaN,NaN,-0.000232036,NaN,NaN,55.2788,55.2786,51.32,51.3213,106.6,51.8562,14.7578,100.0,0.08,0.6,0.27,1.0,0.000232036,-4.19755e-6,4.19755e-6,-6.96332e-13
5,37,1.0,37,firm_value,0.02,0.0185,0.0015,0.0025,NaN,NaN,0.15,1.0,4.52078,54.7839,60.797,NaN,NaN,-0.000126083,NaN,NaN,54.7839,54.7838,49.2126,49.2135,103.997,52.6781,8.84052,100.0,0.08,0.6,0.27,1.0,0.000126083,-2.30146e-6,2.30146e-6,1.35003e-12
6,41,1.0,41,firm_value,0.02,0.018,0.002,0.0025,NaN,NaN,0.15,1.0,4.48157,54.3064,61.2106,NaN,NaN,-0.000142389,NaN,NaN,54.3063,54.3062,47.091,47.0911,101.397,53.5578,3.058,100.0,0.08,0.6,0.27,1.0,0.000142389,-2.62195e-6,2.62195e-6,1.19016e-12


In [12]:
sfeqdf = EqFinDiff.eq_fd(jf.sf; 
                 vbl=jf.sf.optKS.vbl,
                 mu_b=jf.sf.optKS.mu_b,
                 m=jf.sf.optKS.m,
                 c=jf.sf.optKS.c,
                 p=jf.sf.optKS.p)

Computing Equity Vmax
Vmax: 125.0
phi0: -42.25281878230405
phi1: 0.9874999999999998
pv_rfdebt: 56.12855503773488
debt_abs_per_diff: NaN
eq_abs_per_diff: NaN
Equity Vmax: 131.25
 
eq_max: 86.89547575560721
eq_vbl: 0.0
Shape of Gamma matrix: (1498,)
Computing equity values... 
V0: 100.0; equity: 54.382069558969846
Equity Core Function Computation Time: 0.035884857177734375
Total Equity FD Core Function Computation Time: 0.07619690895080566
Total computation time: 0.0820319652557373


,MBR,V0,alpha,c,debt,delta,eq_deriv,eq_deriv_min_val,eq_min_val,eq_negative,eq_vb,equity,firm_value,gross_delta,iota,kappa,lambda,leverage,m,mu_b,p,pi,r,sigmah,sigmal,vb,xi
,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Bool,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64
1,23.9564,100.0,0.6,4.63104,56.1281,0.01975,0.000224516,0.000224516,0.000277442,0,0.0,54.3821,110.51,0.02,0.00025,0.0025,NaN,50.79,1.0,1.0,56.1283,0.27,0.08,NaN,0.15,59.9775,1.0


In [5]:
sfeqdf = EqFinDiff.eq_fd(jf.sf; 
                 vbl=jf.sf.optKS.vbl,
                 mu_b=jf.sf.optKS.mu_b,
                 m=jf.sf.optKS.m,
                 c=jf.sf.optKS.c,
                 p=jf.sf.optKS.p)

rfeqdf = EqFinDiff.eq_fd(jf.rf; 
                 vbl=jf.rf.optKS.vbl,
                 mu_b=jf.rf.optKS.mu_b,
                 m=jf.rf.optKS.m,
                 c=jf.rf.optKS.c,
                 p=jf.rf.optKS.p)

# Make sure firm would choose to hedge the risk under Full Information
println(string("Hedging is optimal under Full Info: ", 
        sfeqdf[1, :firm_value] > rfeqdf[1, :firm_value]))

Computing Equity Vmax
Vmax: 125.0
phi0: -42.25281878230405
phi1: 0.9874999999999998
pv_rfdebt: 56.12855503773488
debt_abs_per_diff: NaN
eq_abs_per_diff: NaN
Equity Vmax: 131.25
 
eq_max: 86.89547575560721
eq_vbl: 0.0
Shape of Gamma matrix: (1498,)
Computing equity values... 
V0: 100.0; equity: 54.382069558969846
Equity Core Function Computation Time: 0.3963630199432373
Total Equity FD Core Function Computation Time: 0.5975401401519775
Total computation time: 1.0986590385437012
Computing Equity Vmax
Vmax: 125.0
phi0: -37.24038376911061
phi1: 0.9999999999999998
pv_rfdebt: 49.481169674042505
debt_abs_per_diff: 1.2711143343120268e-8
eq_abs_per_diff: 0.0008924311407366917
Equity Vmax: 259.86602242642107
 
eq_max: 222.62138379028963
eq_vbl: 0.0
Computing Constant Volatility Equity Values
Finished computing Constant Volatility Equity Values
Time to compute Constant Volatility Equity Values: 0.08295488357543945
 
Shape of Gamma matrix: (1498,)
Computing equity values... 
V0: 100.0; equity: 59.

In [54]:
sfeqdf

,MBR,V0,alpha,c,debt,delta,eq_deriv,eq_deriv_min_val,eq_min_val,eq_negative,eq_vb,equity,firm_value,gross_delta,iota,kappa,lambda,leverage,m,mu_b,p,pi,r,sigmah,sigmal,vb,xi
,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Bool,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64
1,23.9564,100.0,0.6,4.63104,56.1281,0.01975,0.000224516,0.000224516,0.000277442,0,0.0,54.3821,110.51,0.02,0.00025,0.0025,NaN,50.79,1.0,1.0,56.1283,0.27,0.08,NaN,0.15,59.9775,1.0


In [53]:
rfeqdf

,MBR,V0,alpha,c,debt,delta,eq_deriv,eq_deriv_min_val,eq_min_val,eq_negative,eq_vb,equity,firm_value,gross_delta,iota,kappa,lambda,leverage,m,mu_b,p,pi,r,sigmah,sigmal,vb,xi
,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Bool,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64
1,18.3068,100.0,0.6,4.08506,49.4812,0.02,-0.0002035,-0.0002035,0.000912557,0,0.0,59.7672,109.248,0.02,0.0,0.0025,0.2,45.2924,1.0,1.0,49.4797,0.27,0.08,0.2,0.15,53.9347,1.0


In [22]:
# Set joint capital structure to the capital structure of the "true" safe firm
jf.jks.mu_b = jf.sf.optKS.mu_b
jf.jks.m = jf.sf.optKS.m
jf.jks.c = jf.sf.optKS.c
jf.jks.p = jf.sf.optKS.p

# Set measure of "true" safe firm to 1
jf.jks.mu_s = 1.0

jf.jks.vbl= jf.sf.optKS.vbl

59.97749290522221

In [32]:
for modl in modls
    include(string(joinpath(module_path, modl), ".jl"))
end

PyPlot.Figure(PyObject <Figure size 400x100 with 1 Axes>)

In [44]:
JointEq.joint_eq_fd(jf, vbl=60.55, cov_gross_delta=.02)

Computing Equity Vmax
Vmax: 125.0
phi0: -42.25281878230405
phi1: 0.9999999999999998
pv_rfdebt: 56.12855503773488
debt_abs_per_diff: 4.287978188185051e-10
eq_abs_per_diff: 0.000816094929542013
Equity Vmax: 300.8274042113857
 
eq_max: 258.57027738888667
eq_vbl: 0.0
Computing Constant Volatility Equity Values
Finished computing Constant Volatility Equity Values
Time to compute Constant Volatility Equity Values: 0.008044004440307617
 
Shape of Gamma matrix: (1498,)
Computing equity values... 
V0: 100.0; equity: 53.132620204251545
Equity Core Function Computation Time: 0.03602790832519531
Total Equity FD Core Function Computation Time: 0.07874107360839844
Total computation time: 0.10961389541625977
Computing Equity Vmax
Vmax: 125.0
phi0: -42.25281878230405
phi1: 0.9999999999999998
pv_rfdebt: 56.12855503773488
debt_abs_per_diff: 4.287978188185051e-10
eq_abs_per_diff: 0.000816094929542013
Equity Vmax: 300.8274042113857
 
eq_max: 258.57027738888667
eq_vbl: 0.0
Computing Constant Volatility Equ

Computing equity values... 
V0: 100.0; equity: 52.33863810556054
Equity Core Function Computation Time: 0.03596806526184082
Total Equity FD Core Function Computation Time: 0.07854104042053223
Total computation time: 0.10978007316589355
Computing Equity Vmax
Vmax: 125.0
phi0: -42.25281878230405
phi1: 0.9999999999999998
pv_rfdebt: 56.12855503773488
debt_abs_per_diff: 4.287978188185051e-10
eq_abs_per_diff: 0.000816094929542013
Equity Vmax: 300.8274042113857
 
eq_max: 258.57027738888667
eq_vbl: 0.0
Computing Constant Volatility Equity Values
Finished computing Constant Volatility Equity Values
Time to compute Constant Volatility Equity Values: 0.00816798210144043
 
Shape of Gamma matrix: (1498,)
Computing equity values... 
V0: 100.0; equity: 52.121305364991926
Equity Core Function Computation Time: 0.03336691856384277
Total Equity FD Core Function Computation Time: 0.07915210723876953
Total computation time: 0.11040616035461426
Computing Equity Vmax
Vmax: 125.0
phi0: -42.25281878230405
phi

,mu_s,m,mu_b,c,p,fi_vb,sf_vb,rf_vb,vb,eq_deriv,eq_deriv_min_val,eq_min_val,eq_negative,eq_vb,MBR,debt,equity,firm_value,leverage,iota,lambda,sigmah,gross_delta,delta,kappa,sigmal,V0,xi,r,alpha,pi
,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Bool,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64
1,1.0,1.0,1.0,4.63104,56.1283,59.9775,59.9775,NaN,60.55,0.293625,0.293625,0.106471,0,0.0864466,23.9563,56.1281,54.382,110.51,50.79,0.00025,NaN,NaN,0.02,0.01975,0.0025,0.15,100.0,1.0,0.08,0.6,0.27
2,1.0,1.0,1.0,4.63104,56.1283,61.1853,NaN,61.1853,60.55,0.00646249,0.00646249,0.0014713,0,0.0,20.5616,56.1281,52.8927,109.021,51.4838,0.0,0.2,0.2,0.02,0.02,0.0025,0.15,100.0,1.0,0.08,0.6,0.27


In [47]:
JointEq.joint_eq_fd(jf, vbl=60.25, cov_gross_delta=.01)

Computing Equity Vmax
Vmax: 125.0
phi0: -42.25281878230405
phi1: 0.9999999999999998
pv_rfdebt: 56.12855503773488
debt_abs_per_diff: 4.287978188185051e-10
eq_abs_per_diff: 0.000816094929542013
Equity Vmax: 300.8274042113857
 
eq_max: 258.57027738888667
eq_vbl: 0.0
Computing Constant Volatility Equity Values
Finished computing Constant Volatility Equity Values
Time to compute Constant Volatility Equity Values: 0.008043050765991211
 
Shape of Gamma matrix: (1498,)
Computing equity values... 
V0: 100.0; equity: 53.132620204251545
Equity Core Function Computation Time: 0.03811001777648926
Total Equity FD Core Function Computation Time: 0.08379697799682617
Total computation time: 0.11470699310302734
Computing Equity Vmax
Vmax: 125.0
phi0: -42.25281878230405
phi1: 0.9999999999999998
pv_rfdebt: 56.12855503773488
debt_abs_per_diff: 4.287978188185051e-10
eq_abs_per_diff: 0.000816094929542013
Equity Vmax: 300.8274042113857
 
eq_max: 258.57027738888667
eq_vbl: 0.0
Computing Constant Volatility Equ

Computing equity values... 
V0: 100.0; equity: 52.33863810556054
Equity Core Function Computation Time: 0.033384084701538086
Total Equity FD Core Function Computation Time: 0.07914495468139648
Total computation time: 0.11043405532836914
Computing Equity Vmax
Vmax: 125.0
phi0: -42.25281878230405
phi1: 0.9999999999999998
pv_rfdebt: 56.12855503773488
debt_abs_per_diff: 4.287978188185051e-10
eq_abs_per_diff: 0.000816094929542013
Equity Vmax: 300.8274042113857
 
eq_max: 258.57027738888667
eq_vbl: 0.0
Computing Constant Volatility Equity Values
Finished computing Constant Volatility Equity Values
Time to compute Constant Volatility Equity Values: 0.008162975311279297
 
Shape of Gamma matrix: (1498,)
Computing equity values... 
V0: 100.0; equity: 52.121305364991926
Equity Core Function Computation Time: 0.07218694686889648
Total Equity FD Core Function Computation Time: 0.11486101150512695
Total computation time: 0.1461200714111328
Computing Equity Vmax
Vmax: 125.0
phi0: -42.25281878230405
ph

,mu_s,m,mu_b,c,p,fi_vb,sf_vb,rf_vb,vb,eq_deriv,eq_deriv_min_val,eq_min_val,eq_negative,eq_vb,MBR,debt,equity,firm_value,leverage,iota,lambda,sigmah,gross_delta,delta,kappa,sigmal,V0,xi,r,alpha,pi
,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Bool,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64
1,1.0,1.0,1.0,4.63104,56.1283,59.9775,59.9775,NaN,60.25,0.144507,0.144507,0.0305221,0,0.0201022,23.9563,56.1281,54.382,110.51,50.79,0.00025,NaN,NaN,0.02,0.01975,0.0025,0.15,100.0,1.0,0.08,0.6,0.27
2,1.0,1.0,1.0,4.63104,56.1283,61.1853,NaN,61.1853,60.25,-0.00744449,-0.00744449,0.00061959,0,0.0,22.6263,56.1281,53.7985,109.927,51.0596,0.0,0.2,0.2,0.02,0.02,0.0025,0.15,100.0,1.0,0.08,0.6,0.27


In [62]:
JointEq.joint_eq_fd(jf, vbl=61.1, cov_gross_delta=.04)

Computing Equity Vmax
Vmax: 125.0
phi0: -42.25281878230405
phi1: 0.9999999999999998
pv_rfdebt: 56.12855503773488
debt_abs_per_diff: 4.287978188185051e-10
eq_abs_per_diff: 0.000816094929542013
Equity Vmax: 300.8274042113857
 
eq_max: 258.57027738888667
eq_vbl: 0.0
Computing Constant Volatility Equity Values
Finished computing Constant Volatility Equity Values
Time to compute Constant Volatility Equity Values: 0.008047103881835938
 
Shape of Gamma matrix: (1498,)
Computing equity values... 
V0: 100.0; equity: 53.132620204251545
Equity Core Function Computation Time: 0.03627490997314453
Total Equity FD Core Function Computation Time: 0.07952117919921875
Total computation time: 0.1106419563293457
Computing Equity Vmax
Vmax: 125.0
phi0: -42.25281878230405
phi1: 0.9999999999999998
pv_rfdebt: 56.12855503773488
debt_abs_per_diff: 4.287978188185051e-10
eq_abs_per_diff: 0.000816094929542013
Equity Vmax: 300.8274042113857
 
eq_max: 258.57027738888667
eq_vbl: 0.0
Computing Constant Volatility Equi

Computing equity values... 
V0: 100.0; equity: 52.33863810556054
Equity Core Function Computation Time: 0.03603100776672363
Total Equity FD Core Function Computation Time: 0.07892608642578125
Total computation time: 0.11038589477539062
Computing Equity Vmax
Vmax: 125.0
phi0: -42.25281878230405
phi1: 0.9999999999999998
pv_rfdebt: 56.12855503773488
debt_abs_per_diff: 4.287978188185051e-10
eq_abs_per_diff: 0.000816094929542013
Equity Vmax: 300.8274042113857
 
eq_max: 258.57027738888667
eq_vbl: 0.0
Computing Constant Volatility Equity Values
Finished computing Constant Volatility Equity Values
Time to compute Constant Volatility Equity Values: 0.008173942565917969
 
Shape of Gamma matrix: (1498,)
Computing equity values... 
V0: 100.0; equity: 52.121305364991926
Equity Core Function Computation Time: 0.033468008041381836
Total Equity FD Core Function Computation Time: 0.08014488220214844
Total computation time: 0.11163997650146484
Computing Equity Vmax
Vmax: 125.0
phi0: -42.25281878230405
p

,mu_s,m,mu_b,c,p,fi_vb,sf_vb,rf_vb,vb,eq_deriv,eq_deriv_min_val,eq_min_val,eq_negative,eq_vb,MBR,debt,equity,firm_value,leverage,iota,lambda,sigmah,gross_delta,delta,kappa,sigmal,V0,xi,r,alpha,pi
,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Bool,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64
1,1.0,1.0,1.0,4.63104,56.1283,59.9775,59.9775,NaN,61.1,0.539184,0.539184,0.353574,0,0.317648,23.9563,56.1281,54.382,110.51,50.79,0.00025,NaN,NaN,0.02,0.01975,0.0025,0.15,100.0,1.0,0.08,0.6,0.27
2,1.0,1.0,1.0,4.63104,56.1283,61.1853,NaN,61.1853,61.1,-0.00249503,-0.00249503,0.000807846,0,0.0,15.3884,56.1281,50.6231,106.751,52.5784,0.0,0.2,0.2,0.02,0.02,0.0025,0.15,100.0,1.0,0.08,0.6,0.27


In [88]:
?ModelObj.set_opt_k_struct(jf.rf, )

No documentation found.

`Main.ModelObj.set_opt_k_struct` is a `Function`.

```
# 1 method for generic function "set_opt_k_struct":
[1] set_opt_k_struct(mobj, df::DataFrame; m, tol) in Main.ModelObj at /home/artur/BondPricing/Julia/modules/ModelObj.jl:207
```


In [89]:
?Batch.optimal_cvm_capital_struct()

No documentation found.

`Main.Batch.optimal_cvm_capital_struct` is a `Function`.

```
# 1 method for generic function "optimal_cvm_capital_struct":
[1] optimal_cvm_capital_struct(bt, cvm; firm_obj_fun, df, dfname, interp_polyOrder, filter_windowSize, filter_polyOrder, replace_optdf, save_optdf, optdf_name) in Main.Batch at /home/artur/BondPricing/Julia/modules/Batch.jl:2442
```


In [80]:
cvm_m=1.
obj_fun=:firm_value
fig_dpi=400
fig_aspect=.55
graph_format="png"
return_fig=false

bt = Batch.BatchObj()

pt = ModelPlots.PlotsObj(bt; firm_obj_fun=obj_fun, cvm_m=cvm_m, svm_m =1.)

Loading optimal results dataframe...
Setting parameter dictionary using (m, m_comb_num) ID pair...
Loading optimal results dataframe...


Main.ModelPlots.PlotStruct
  bt: Main.Batch.BatchStruct
  cvm_data: DataFrame
  svm_data: DataFrame
  obj_fun: Symbol firm_value
  _svm_surf: DataFrame
  xylabels: Dict{Symbol,Array{String,1}}
  zlabels: Dict{Symbol,Array{String,1}}


In [83]:
pt.svm_data.debt_diff

150-element CSV.Column{Float64,Float64}:
  0.0021859145322196127 
  0.001483337037534227  
  0.0012126351102637045 
  0.0007301098337393341 
  0.0003081050683917397 
  0.0018822423828268597 
  0.0014508210429866608 
  0.0012844246040231155 
  0.0010271775903021307 
  0.00037399335938914646
  0.00015083964150619522
  2.5687520761152882e-5 
 -6.116148227874874e-7  
 -1.821202801011168e-6  
  0.00030967790491587266
 -0.005562627321374691  
 -0.004725135946884507  
 -0.0030817533313225454 
 -0.0008792684710243748 
  0.022822747245569985  
 -0.011066514543529138  
  0.055412180750195716  
  0.013948333787460854  
  0.005145606023383209  
  0.004550300971267518  
  0.0010574163371472878 
  0.0009137664346781094 
  0.0008976178028348158 
  0.0007777581883416929 
  0.0004788070830414881 
 -0.006073801900029707  
 -0.004641309978865138  
 -0.004558628675511045  
 -0.0036517969939993122 
 -0.0027214976447069716 
 -0.008272079106490082  
 -0.0008209956882154756 
  0.006415790330272841  
 -0.00919

,comb_num,m,m_comb_num,obj_fun,gross_delta,delta,iota,kappa
,Int64,Float64,Int64,String,Float64,Float64,Float64,Float64
1,1,1.0,1,firm_value,0.02,0.02,0.0,0.0025
2,5,1.0,5,firm_value,0.02,0.01995,5.0e-5,0.0025
3,9,1.0,9,firm_value,0.02,0.0199,0.0001,0.0025
4,13,1.0,13,firm_value,0.02,0.01985,0.00015,0.0025
5,17,1.0,17,firm_value,0.02,0.0198,0.0002,0.0025
6,21,1.0,21,firm_value,0.02,0.01975,0.00025,0.0025
7,25,1.0,25,firm_value,0.02,0.0195,0.0005,0.0025
8,29,1.0,29,firm_value,0.02,0.01925,0.00075,0.0025
9,33,1.0,33,firm_value,0.02,0.019,0.001,0.0025


,comb_num,m,m_comb_num,mu_b,xi,kappa,gross_delta,lambda
,Int64,Float64,Int64,Float64,Float64,Float64,Float64,Float64
1,1,1.0,1,1.0,1.0,0.0025,0.02,NaN
2,2,1.0,2,1.0,1.0,0.005,0.02,NaN
3,3,1.0,3,1.0,1.0,0.01,0.02,NaN
4,4,1.0,4,1.0,1.0,0.015,0.02,NaN
5,5,1.0,5,1.0,1.0,0.0025,0.02,NaN
6,6,1.0,6,1.0,1.0,0.005,0.02,NaN
7,7,1.0,7,1.0,1.0,0.01,0.02,NaN
8,8,1.0,8,1.0,1.0,0.015,0.02,NaN
9,9,1.0,9,1.0,1.0,0.0025,0.02,NaN


In [14]:
jf.rf.pm.sigmah

0.225

In [17]:
jf.rf.mu_b = 1.
jf.rf.c=3.
jf.rf.p=63.9

63.9

In [41]:
jf.rf.pm.gross_delta = .02

0.02

In [51]:
jf.sf.pm.gross_delta
jf.sf.pm.iota = 

0.00025

In [49]:
jf.sf.mu_b = 1.
jf.sf.c=3.
jf.sf.p=63.9
vbl = AnalyticFunctions.get_cvm_vb(jf.sf, jf.sf.pm.sigmal)

70.474336584049

In [48]:
BondPrInterp.get_cvm_debt_price(jf.sf, vbl, jf.sf.pm.sigmal, )

NaN

In [73]:
jf.rf.pm.gross_delta = 0.00005

5.0e-5

In [74]:
AnalyticFunctions.get_cvm_vb(jf.rf, jf.rf.pm.sigmal)

67.37109333245587

In [75]:
EqFinDiff.get_cvm_eq(jf.rf, jf.rf.pm.V0, jf.rf.pm.sigmal; )

43.25873980133952

In [67]:
jf.rf.pm.gross_delta = 0.005

0.005

In [68]:
AnalyticFunctions.get_cvm_vb(jf.rf, jf.rf.pm.sigmal)

67.98104086064232

In [69]:
EqFinDiff.get_cvm_eq(jf.rf, jf.rf.pm.V0, jf.rf.pm.sigmal; )

42.74396525565551

In [59]:
jf.rf.pm.gross_delta = 0.05

0.05

In [60]:
AnalyticFunctions.get_cvm_vb(jf.rf, jf.rf.pm.sigmal)

74.19455147464774

In [61]:
EqFinDiff.get_cvm_eq(jf.rf, jf.rf.pm.V0, jf.rf.pm.sigmal; )

32.72523057667835

In [62]:
jf.rf.pm.gross_delta = .02

0.02

In [63]:
AnalyticFunctions.get_cvm_vb(jf.rf, jf.rf.pm.sigmah)

69.9911569118274

In [64]:
EqFinDiff.get_cvm_eq(jf.rf, jf.rf.pm.V0, jf.rf.pm.sigmah; )

31.611229455002746

In [65]:
jf.rf.pm.gross_delta = .03

0.03

In [66]:
AnalyticFunctions.get_cvm_vb(jf.rf, jf.rf.pm.sigmah)

70.74598032600049

In [55]:
EqFinDiff.get_cvm_eq(jf.rf, jf.rf.pm.V0, jf.rf.pm.sigmah; )

29.804285594357925

In [8]:
?AnalyticFunctions.cvm_eq()

No documentation found.

`Main.AnalyticFunctions.cvm_eq` is a `Function`.

```
# 1 method for generic function "cvm_eq":
[1] cvm_eq(v, mu_b, m, c, p, sigma, r, gross_delta, iota, xi, k, alpha, pi) in Main.AnalyticFunctions at /home/artur/BondPricing/Julia/modules/AnalyticFunctions.jl:186
```
